In [1]:
#pip install tensorflow

In [2]:
import os
import numpy as np
import cv2
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [3]:
directory = "TImages/"

In [4]:
image_size = (244, 244)
classes = ["0", "1", "2"]

In [5]:
# Create an ImageDataGenerator instance with data augmentation settings
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    samplewise_center=True,
    samplewise_std_normalization=True,
)

In [6]:
training_data = []

def create_training_data():
    for category in classes:
        path = os.path.join(directory, category)
        class_num = classes.index(category)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, image_size)

                # Generate and store augmented images
                augmented_images = []
                augmented_images.append(new_array)  # Original image
                img_array_aug = new_array.reshape((1,) + new_array.shape)
                i = 0
                for batch in datagen.flow(img_array_aug, batch_size=1):
                    augmented_images.append(batch[0])
                    i += 1
                    if i >= 5:  # Generate 5 augmented images per input image
                        break

                for augmented_image in augmented_images:
                    image_hsv = cv2.cvtColor(augmented_image, cv2.COLOR_BGR2HSV)
                    training_data.append([image_hsv, class_num])

            except Exception as e:
                pass

In [7]:
create_training_data()

In [8]:
lenofimage = len(training_data)
print(lenofimage)

5850


In [9]:
X=[]
y=[]

for categories, label in training_data:
    X.append(categories)
    y.append(label)
X= np.array(X).reshape(lenofimage,-1)

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, )

In [12]:
## KNN MODEL

In [12]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [14]:
# Feature Scaling
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [13]:
# Define the parameter grid for hyperparameter tuning
param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11]  # You can adjust this range
}

# Create the KNN model
knn = KNeighborsClassifier(n_neighbors = 5, weights='uniform', p = 2, algorithm='auto') # metric = 'minkowski',

In [14]:
# Create the GridSearchCV object
grid_search = GridSearchCV(knn, param_grid, cv=5, verbose=2)

In [15]:
# Fit the GridSearchCV object to your data
grid_search.fit(X_train, y_train)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] END ......................................n_neighbors=3; total time=  17.0s
[CV] END ......................................n_neighbors=3; total time=  15.4s
[CV] END ......................................n_neighbors=3; total time=  15.0s
[CV] END ......................................n_neighbors=3; total time=  14.3s
[CV] END ......................................n_neighbors=3; total time=  14.1s
[CV] END ......................................n_neighbors=5; total time=  13.8s
[CV] END ......................................n_neighbors=5; total time=  13.6s
[CV] END ......................................n_neighbors=5; total time=  13.4s
[CV] END ......................................n_neighbors=5; total time=  13.7s
[CV] END ......................................n_neighbors=5; total time=  13.3s
[CV] END ......................................n_neighbors=7; total time=  13.0s
[CV] END ......................................n_

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': [3, 5, 7, 9, 11]}, verbose=2)

In [16]:
# Get the best estimator and evaluate it
best_knn = grid_search.best_estimator_
y_test_pred = best_knn.predict(X_test)
accuracy = accuracy_score(y_test, y_test_pred)
print("Best KNN Parameters:", grid_search.best_params_)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Best KNN Parameters: {'n_neighbors': 3}
Accuracy: 93.25%


In [17]:
df = pd.DataFrame(grid_search.cv_results_)
df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.872253,0.217112,13.386359,0.937131,3,{'n_neighbors': 3},0.941239,0.943376,0.942308,0.946581,0.945513,0.943803,0.001982,1
1,1.661447,0.024543,11.998944,0.186413,5,{'n_neighbors': 5},0.945513,0.942308,0.941239,0.944444,0.939103,0.942521,0.002281,2
2,3.953403,1.916407,18.755096,5.297885,7,{'n_neighbors': 7},0.943376,0.940171,0.940171,0.944444,0.941239,0.941880,0.001736,3
3,5.510572,0.027900,23.054781,0.260629,9,{'n_neighbors': 9},0.944444,0.939103,0.942308,0.940171,0.941239,0.941453,0.001838,4
4,5.632220,0.220958,23.003177,0.411480,11,{'n_neighbors': 11},0.940171,0.938034,0.940171,0.939103,0.940171,0.939530,0.000855,5


In [20]:
df[['param_n_neighbors', 'params', 'mean_test_score']]

,param_n_neighbors,params,mean_test_score
0,3,{'n_neighbors': 3},0.943803
1,5,{'n_neighbors': 5},0.942521
2,7,{'n_neighbors': 7},0.941880
3,9,{'n_neighbors': 9},0.941453
4,11,{'n_neighbors': 11},0.939530


In [21]:
grid_search.best_score_

0.9438034188034188

In [22]:
grid_search.best_params_

{'n_neighbors': 3}